In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
from sqlalchemy import create_engine, inspect
from utils.database_utils import *
from utils.config_utils import *
from constants import *

db_config = get_db_config(read_config(CONFIG_FILE_PATH))
# Create SQLAlchemy engine
engine = connect(db_config)

# Connect and Inspect
inspector = inspect(engine)

# Get table names
table_names = inspector.get_table_names()
table_names

Connection to PostgreSQL successful


['routes_table',
 'routes_weather',
 'drivers_table',
 'trucks_table',
 'city_weather',
 'truck_schedule_table',
 'traffic_table']

In [10]:
import hopsworks
import pandas as pd

project = hopsworks.login(api_key_value = 'S2PoqNKdqdQ2EXNW.hybyaHPCvA5Jx6uoeL55fG2X5u3ogs3yUxcPmkGUKOc6DsNapeRmD3L6PlpefmOz')
fs = project.get_feature_store()

feature_groups = [table + '_fg' for table in table_names]

feature_dataframes = {}

# Loop through each feature group and fetch the data
for fg_name in feature_groups:
    # Get the feature group object
    fg = fs.get_feature_group(fg_name, version=2)  # Ensure you use the correct version
    
    # Fetch the data as a Pandas DataFrame
    query = fg.select_all()
    df=query.read(read_options={"use_hive": True})
    # df = fg.read(read_options={"use_hive": True})
    
    # Store in dictionary
    feature_dataframes[fg_name] = df

# Now you have a dictionary of DataFrames, where the key is the feature group name
print(feature_dataframes)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1022104
Connected. Call `.close()` to terminate connection gracefully.
2024-09-28 01:14:22,473 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (1.12s) 
2024-09-28 01:14:25,059 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (27.24s) 
2024-09-28 01:14:53,597 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (1.05s) 
2024-09-28 01:14:56,066 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (0.95s) 
2024-09-28 01:14:57,976 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (3.79s) 
2024-09-28 01:15:02,977 WARNING: DeprecationWarning: s

In [11]:
for key, df in feature_dataframes.items():
    if 'event_time' in df.columns:
        df.drop(columns=['event_time'], inplace=True)

In [12]:
city_weather_df = feature_dataframes['city_weather_fg']
drivers_df  = feature_dataframes['drivers_table_fg']
routes_df  = feature_dataframes['routes_table_fg']
routes_weather_df  = feature_dataframes['routes_weather_fg']
traffic_df  = feature_dataframes['traffic_table_fg']
truck_schedule_df  = feature_dataframes['truck_schedule_table_fg']
trucks_df  = feature_dataframes['trucks_table_fg']

In [13]:
city_weather_df

,id,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,52494,C-841ebdcb,2019-01-27,500,55,5,Moderate or heavy rain shower,0.0,84,5,1022,0,0,0,0
1,26247,C-f5ed4c15,2019-02-06,1400,5,6,Sunny,0.0,80,6,1026,0,0,0,0
2,7966,C-731988ba,2019-01-10,2100,46,3,Clear,0.0,57,6,1016,0,0,0,0
3,31016,C-9bbf5d8a,2019-01-06,700,72,6,Sunny,0.0,61,6,1014,0,0,0,0
4,11190,C-c7cacd1d,2019-01-07,500,21,8,Cloudy,0.0,60,6,1033,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52071,40378,C-2aaf0e1a,2019-01-28,900,19,2,Sunny,0.0,66,6,1021,0,0,0,0
52072,18331,C-e5bfb4e5,2019-01-28,1800,1,10,Cloudy,0.0,90,6,1015,0,0,0,0
52073,5153,C-451776b7,2019-01-31,1600,57,8,Partly cloudy,0.0,81,6,1013,0,0,0,0
52074,18404,C-e5bfb4e5,2019-01-31,1900,-9,3,Cloudy,0.0,65,6,1028,0,0,0,0


# Preparation

## Drop duplicates

In [14]:
city_weather_df = city_weather_df.drop_duplicates(subset=['city_id', 'date', 'hour'], inplace=False)

routes_weather_df = routes_weather_df.drop_duplicates(subset=['route_id', 'date'], inplace=False)

trucks_df = trucks_df.drop_duplicates(subset=['truck_id'], inplace=False)

drivers_df = drivers_df.drop_duplicates(subset=['driver_id'], inplace=False)

routes_df = routes_df.drop_duplicates(subset=['route_id', 'destination_id', 'origin_id'], inplace=False)

truck_schedule_df = truck_schedule_df.drop_duplicates(subset=['truck_id', 'route_id', 'departure_date'], inplace=False)

traffic_df = traffic_df.drop_duplicates(subset=['route_id', 'date', 'hour'])

## Drop unnecesary columns

In [15]:
# Drop unneccesary columms
city_weather_df = city_weather_df.drop(columns=['chanceofrain','chanceoffog','chanceofsnow','chanceofthunder'])

routes_weather_df = routes_weather_df.drop(columns=['chanceofrain','chanceoffog','chanceofsnow','chanceofthunder'])

## Change to date_time column

In [16]:
city_weather_df['date_time'] = city_weather_df['date'] + pd.to_timedelta(city_weather_df['hour'] // 100, unit='h')

In [17]:
city_weather_df = city_weather_df.drop(columns=['date','hour'])
city_weather_df.insert(1, 'date_time', city_weather_df.pop('date_time'))

In [18]:
traffic_df['date_time'] = traffic_df['date'] + pd.to_timedelta(traffic_df['hour'] // 100, unit='h')

2024-09-28 01:17:27,525 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
traffic_df = traffic_df.drop(columns=['date','hour'])
traffic_df.insert(1, 'date_time', traffic_df.pop('date_time'))

In [20]:
traffic_df

,id,date_time,route_id,no_of_vehicles,accident,time_period
0,1822754,2019-02-09 16:00:00,R-1a963262,2394.0,0,Late Afternoon
1,1074812,2019-01-26 19:00:00,R-24bd80c9,2589.0,0,Evening
2,1225804,2019-01-16 03:00:00,R-100ed4e5,652.0,0,Early Morning
3,414810,2019-02-03 17:00:00,R-ce2410a8,2296.0,0,Late Afternoon
4,2576420,2019-01-25 10:00:00,R-97e6b1e3,1827.0,0,Late Morning
...,...,...,...,...,...,...
2597908,156170,2019-01-22 01:00:00,R-c2361128,642.0,0,Late Night
2597909,728441,2019-02-07 16:00:00,R-e6a70c54,746.0,1,Late Afternoon
2597910,2175704,2019-01-26 22:00:00,R-8874a6c1,2245.0,0,Night
2597911,2053819,2019-01-08 09:00:00,R-35ca9198,2370.0,0,Late Morning


# Feature Engineering

## Truck schedule and routes weather merge table

In [21]:
truck_schedule_df_fe = truck_schedule_df.copy()

In [22]:
truck_schedule_df_fe['departure_date'] = truck_schedule_df_fe['departure_date'].dt.floor('6H') 
truck_schedule_df_fe['estimated_arrival'] = truck_schedule_df_fe['estimated_arrival'].dt.ceil('6H') 
truck_schedule_df_fe

,id,truck_id,route_id,departure_date,estimated_arrival,delay
0,8256,25645401,R-8adb758d,2019-02-09 06:00:00,2019-02-09 18:00:00,0
1,9236,27121057,R-9e7ea58a,2019-02-12 06:00:00,2019-02-13 06:00:00,1
2,1932,10966168,R-b2e25936,2019-01-10 06:00:00,2019-01-11 06:00:00,1
3,10256,11835599,R-8e99ef5f,2019-01-31 06:00:00,2019-02-01 18:00:00,1
4,5242,29121693,R-5efde222,2019-01-25 06:00:00,2019-01-25 18:00:00,0
...,...,...,...,...,...,...
12303,10925,63052683,R-86dd80fc,2019-01-09 06:00:00,2019-01-11 12:00:00,1
12304,2971,18678032,R-52653b9a,2019-01-13 06:00:00,2019-01-13 18:00:00,1
12305,9841,28817628,R-9b98b32f,2019-01-16 06:00:00,2019-01-17 18:00:00,1
12306,7157,11759429,R-0bac03b2,2019-02-03 06:00:00,2019-02-04 00:00:00,0


In [23]:
# Step 1: Create a 'date' column by generating date ranges with a frequency of 6 hours
truck_schedule_df_fe['date'] = truck_schedule_df_fe.apply(
    lambda row: pd.date_range(start=row['departure_date'], 
                              end=row['estimated_arrival'], 
                              freq='6H'), axis=1)

# Step 2: Explode the 'date' column to create separate rows for each date in the range
truck_schedule_data_exploded = truck_schedule_df_fe.explode('date')
truck_schedule_data_exploded

,id,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,8256,25645401,R-8adb758d,2019-02-09 06:00:00,2019-02-09 18:00:00,0,2019-02-09 06:00:00
0,8256,25645401,R-8adb758d,2019-02-09 06:00:00,2019-02-09 18:00:00,0,2019-02-09 12:00:00
0,8256,25645401,R-8adb758d,2019-02-09 06:00:00,2019-02-09 18:00:00,0,2019-02-09 18:00:00
1,9236,27121057,R-9e7ea58a,2019-02-12 06:00:00,2019-02-13 06:00:00,1,2019-02-12 06:00:00
1,9236,27121057,R-9e7ea58a,2019-02-12 06:00:00,2019-02-13 06:00:00,1,2019-02-12 12:00:00
...,...,...,...,...,...,...,...
12307,1809,33788665,R-1ec2684e,2019-01-07 06:00:00,2019-01-08 06:00:00,0,2019-01-07 06:00:00
12307,1809,33788665,R-1ec2684e,2019-01-07 06:00:00,2019-01-08 06:00:00,0,2019-01-07 12:00:00
12307,1809,33788665,R-1ec2684e,2019-01-07 06:00:00,2019-01-08 06:00:00,0,2019-01-07 18:00:00
12307,1809,33788665,R-1ec2684e,2019-01-07 06:00:00,2019-01-08 06:00:00,0,2019-01-08 00:00:00


In [24]:
# Step 3: Merge the resulting dataframe with route_weather on 'route_id' and 'date'
merged_df = pd.merge(truck_schedule_data_exploded, routes_weather_df, 
                     how='left', 
                     on=['route_id', 'date'])

In [25]:
routes_weather_df[routes_weather_df['route_id']=='R-266863d8'].sort_values(by = 'date')

,id,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
306007,71439,R-266863d8,2019-01-02 06:00:00,77,6,Sunny,0.0,14,6,1017
96458,71440,R-266863d8,2019-01-03 00:00:00,73,6,Clear,0.0,14,6,1017
257164,71441,R-266863d8,2019-01-03 06:00:00,73,7,Clear,0.0,14,6,1017
86515,293237,R-266863d8,2019-01-03 12:00:00,82,8,Clear,0.0,14,6,1010
345579,293238,R-266863d8,2019-01-03 18:00:00,81,8,Clear,0.0,15,6,1010
...,...,...,...,...,...,...,...,...,...,...
280598,293319,R-266863d8,2019-02-13 12:00:00,77,9,Sunny,0.0,19,6,1011
157766,293320,R-266863d8,2019-02-13 18:00:00,84,10,Sunny,0.0,16,6,1011
212073,71524,R-266863d8,2019-02-14 00:00:00,68,6,Sunny,0.0,22,6,1017
32970,71525,R-266863d8,2019-02-14 06:00:00,72,6,Sunny,0.0,18,6,1017


In [26]:
merged_df[merged_df['id_x']==347]

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,date,id_y,temp,wind_speed,description,precip,humidity,visibility,pressure
9636,347,21093078,R-266863d8,2019-01-01 06:00:00,2019-01-01 12:00:00,0,2019-01-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9637,347,21093078,R-266863d8,2019-01-01 06:00:00,2019-01-01 12:00:00,0,2019-01-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
def custom_mode(x):
    modes = x.mode()
    if not modes.empty:
        return modes.iloc[0]  # Get the most frequent value (mode)
    else:
        return np.nan

In [28]:
truck_schedule_route_weather_df = merged_df.groupby(['id_x', 'truck_id', 'route_id'], as_index=False).agg(
    route_avg_temp=('temp','mean'),
    route_avg_wind_speed=('wind_speed','mean'),
    route_avg_humidity=('humidity','mean'),
    route_avg_pressure=('pressure','mean'),
    route_description=('description', custom_mode)
)

In [29]:
truck_schedule_route_weather_df[truck_schedule_route_weather_df['route_avg_temp'].isna()].head()

,id_x,truck_id,route_id,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_description
346,347,21093078,R-266863d8,NaN,NaN,NaN,NaN,NaN
386,387,18653108,R-c15e9809,NaN,NaN,NaN,NaN,NaN
1477,1478,22112064,R-1a08406e,NaN,NaN,NaN,NaN,NaN
1479,1480,13779072,R-1d00410b,NaN,NaN,NaN,NaN,NaN
1512,1513,16815307,R-3cc18593,NaN,NaN,NaN,NaN,NaN


In [30]:
truck_schedule_route_weather_df = truck_schedule_route_weather_df.rename(columns={'id_x':'id'})

In [31]:
schedule_weather_merge_df=truck_schedule_df.merge(truck_schedule_route_weather_df,on=['id','truck_id','route_id'],how='left')

In [32]:
schedule_weather_merge_df

,id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_description
0,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,76.000000,6.000000,58.000000,1011.666667,Sunny
1,9236,27121057,R-9e7ea58a,2019-02-12 07:00:00,2019-02-13 03:58:48,1,60.200000,5.200000,74.600000,1016.400000,Clear
2,1932,10966168,R-b2e25936,2019-01-10 07:00:00,2019-01-11 03:07:48,1,76.600000,12.000000,65.400000,1015.000000,Sunny
3,10256,11835599,R-8e99ef5f,2019-01-31 07:00:00,2019-02-01 16:31:48,1,NaN,NaN,NaN,NaN,NaN
4,5242,29121693,R-5efde222,2019-01-25 07:00:00,2019-01-25 12:01:48,0,45.666667,7.333333,64.000000,1015.000000,Partly cloudy
...,...,...,...,...,...,...,...,...,...,...,...
12303,10925,63052683,R-86dd80fc,2019-01-09 07:00:00,2019-01-11 08:04:48,1,72.500000,7.200000,62.600000,1012.600000,Clear
12304,2971,18678032,R-52653b9a,2019-01-13 07:00:00,2019-01-13 14:00:36,1,63.000000,11.333333,93.000000,1009.000000,Overcast
12305,9841,28817628,R-9b98b32f,2019-01-16 07:00:00,2019-01-17 16:22:12,1,43.714286,5.714286,84.142857,1019.571429,Clear
12306,7157,11759429,R-0bac03b2,2019-02-03 07:00:00,2019-02-03 20:16:48,0,84.500000,12.000000,45.750000,1009.750000,Sunny


## City weather merge

In [33]:
nearest_hour_schedule_df=truck_schedule_df.copy()
nearest_hour_schedule_df['estimated_arrival_nearest_hour']=nearest_hour_schedule_df['estimated_arrival'].dt.round("H")
nearest_hour_schedule_df['departure_date_nearest_hour']=nearest_hour_schedule_df['departure_date'].dt.round("H")
nearest_hour_schedule_route_df=pd.merge(nearest_hour_schedule_df, routes_df, on='route_id', how='left')


In [34]:
nearest_hour_schedule_route_df

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,id_y,origin_id,destination_id,distance,average_hours
0,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,1659.0,C-b5ff31cd,C-b25a09de,426.02,8.52
1,9236,27121057,R-9e7ea58a,2019-02-12 07:00:00,2019-02-13 03:58:48,1,2019-02-13 04:00:00,2019-02-12 07:00:00,1278.0,C-b25a09de,C-2c9e75ef,1049.19,20.98
2,1932,10966168,R-b2e25936,2019-01-10 07:00:00,2019-01-11 03:07:48,1,2019-01-11 03:00:00,2019-01-10 07:00:00,1433.0,C-40a81eb1,C-825b2625,1006.61,20.13
3,10256,11835599,R-8e99ef5f,2019-01-31 07:00:00,2019-02-01 16:31:48,1,2019-02-01 17:00:00,2019-01-31 07:00:00,303.0,C-c92599e2,C-ff8c0c3c,1676.61,33.53
4,5242,29121693,R-5efde222,2019-01-25 07:00:00,2019-01-25 12:01:48,0,2019-01-25 12:00:00,2019-01-25 07:00:00,730.0,C-ff8c0c3c,C-a9f2c329,251.66,5.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10925,63052683,R-86dd80fc,2019-01-09 07:00:00,2019-01-11 08:04:48,1,2019-01-11 08:00:00,2019-01-09 07:00:00,1944.0,C-d3bb431c,C-f5ed4c15,2453.91,49.08
12304,2971,18678032,R-52653b9a,2019-01-13 07:00:00,2019-01-13 14:00:36,1,2019-01-13 14:00:00,2019-01-13 07:00:00,1091.0,C-280b55fb,C-5d86b887,350.41,7.01
12305,9841,28817628,R-9b98b32f,2019-01-16 07:00:00,2019-01-17 16:22:12,1,2019-01-17 16:00:00,2019-01-16 07:00:00,2019.0,C-90e30162,C-73ae5412,1668.41,33.37
12306,7157,11759429,R-0bac03b2,2019-02-03 07:00:00,2019-02-03 20:16:48,0,2019-02-03 20:00:00,2019-02-03 07:00:00,1911.0,C-b5282c3b,C-d9e9d934,664.12,13.28


In [35]:
origin_weather_data_df = city_weather_df.copy()
destination_weather_data_df = city_weather_df.copy()

In [36]:
origin_weather_data_df

,id,date_time,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure
0,52494,2019-01-27 05:00:00,C-841ebdcb,55,5,Moderate or heavy rain shower,0.0,84,5,1022
1,26247,2019-02-06 14:00:00,C-f5ed4c15,5,6,Sunny,0.0,80,6,1026
2,7966,2019-01-10 21:00:00,C-731988ba,46,3,Clear,0.0,57,6,1016
3,31016,2019-01-06 07:00:00,C-9bbf5d8a,72,6,Sunny,0.0,61,6,1014
4,11190,2019-01-07 05:00:00,C-c7cacd1d,21,8,Cloudy,0.0,60,6,1033
...,...,...,...,...,...,...,...,...,...,...
52071,40378,2019-01-28 09:00:00,C-2aaf0e1a,19,2,Sunny,0.0,66,6,1021
52072,18331,2019-01-28 18:00:00,C-e5bfb4e5,1,10,Cloudy,0.0,90,6,1015
52073,5153,2019-01-31 16:00:00,C-451776b7,57,8,Partly cloudy,0.0,81,6,1013
52074,18404,2019-01-31 19:00:00,C-e5bfb4e5,-9,3,Cloudy,0.0,65,6,1028


In [37]:
nearest_hour_schedule_route_origin_weather_df=pd.merge(nearest_hour_schedule_route_df, origin_weather_data_df, left_on=['departure_date_nearest_hour','origin_id'], right_on=['date_time','city_id'], how='left')

In [38]:
nearest_hour_schedule_route_origin_weather_df = nearest_hour_schedule_route_origin_weather_df.drop(columns='id')

In [39]:
origin_destination_weather_merge_df=pd.merge(nearest_hour_schedule_route_origin_weather_df, destination_weather_data_df, left_on=['estimated_arrival_nearest_hour','destination_id'], right_on=['date_time','city_id'], how='left')

In [40]:
origin_destination_weather_merge_df

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,id_y,origin_id,...,id,date_time_y,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y
0,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,1659.0,C-b5ff31cd,...,29633.0,2019-02-09 16:00:00,C-b25a09de,27.0,7.0,Sunny,0.0,35.0,6.0,1041.0
1,9236,27121057,R-9e7ea58a,2019-02-12 07:00:00,2019-02-13 03:58:48,1,2019-02-13 04:00:00,2019-02-12 07:00:00,1278.0,C-b25a09de,...,34133.0,2019-02-13 04:00:00,C-2c9e75ef,41.0,13.0,Partly cloudy,0.0,47.0,6.0,1023.0
2,1932,10966168,R-b2e25936,2019-01-10 07:00:00,2019-01-11 03:07:48,1,2019-01-11 03:00:00,2019-01-10 07:00:00,1433.0,C-40a81eb1,...,45484.0,2019-01-11 03:00:00,C-825b2625,25.0,6.0,Clear,0.0,60.0,6.0,1028.0
3,10256,11835599,R-8e99ef5f,2019-01-31 07:00:00,2019-02-01 16:31:48,1,2019-02-01 17:00:00,2019-01-31 07:00:00,303.0,C-c92599e2,...,17322.0,2019-02-01 17:00:00,C-ff8c0c3c,41.0,7.0,Partly cloudy,0.0,65.0,4.0,1022.0
4,5242,29121693,R-5efde222,2019-01-25 07:00:00,2019-01-25 12:01:48,0,2019-01-25 12:00:00,2019-01-25 07:00:00,730.0,C-ff8c0c3c,...,10525.0,2019-01-25 12:00:00,C-a9f2c329,7.0,8.0,Light snow,0.0,85.0,6.0,1024.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10925,63052683,R-86dd80fc,2019-01-09 07:00:00,2019-01-11 08:04:48,1,2019-01-11 08:00:00,2019-01-09 07:00:00,1944.0,C-d3bb431c,...,25617.0,2019-01-11 08:00:00,C-f5ed4c15,14.0,3.0,Patchy heavy snow,0.0,86.0,3.0,1023.0
12304,2971,18678032,R-52653b9a,2019-01-13 07:00:00,2019-01-13 14:00:36,1,2019-01-13 14:00:00,2019-01-13 07:00:00,1091.0,C-280b55fb,...,38919.0,2019-01-13 14:00:00,C-5d86b887,30.0,10.0,Heavy snow,0.0,96.0,1.0,1025.0
12305,9841,28817628,R-9b98b32f,2019-01-16 07:00:00,2019-01-17 16:22:12,1,2019-01-17 16:00:00,2019-01-16 07:00:00,2019.0,C-90e30162,...,2609.0,2019-01-17 16:00:00,C-73ae5412,55.0,15.0,Patchy rain possible,0.0,69.0,6.0,1019.0
12306,7157,11759429,R-0bac03b2,2019-02-03 07:00:00,2019-02-03 20:16:48,0,2019-02-03 20:00:00,2019-02-03 07:00:00,1911.0,C-b5282c3b,...,42741.0,2019-02-03 20:00:00,C-d9e9d934,46.0,4.0,Patchy light rain,0.0,86.0,6.0,1019.0


## Traffic table merge

In [282]:
truck_schedule_for_traffic_df = truck_schedule_df.copy()

In [283]:
truck_schedule_for_traffic_df['estimated_arrival_nearest_hour']=truck_schedule_for_traffic_df['estimated_arrival'].dt.round("H")
truck_schedule_for_traffic_df['departure_date_nearest_hour']=truck_schedule_for_traffic_df['departure_date'].dt.round("H")

In [284]:
hourly_exploded_scheduled_df=(truck_schedule_for_traffic_df.assign(custom_date = [pd.date_range(start, end, freq='H')  # Create custom date ranges
for start, end
in zip(truck_schedule_for_traffic_df['departure_date'], truck_schedule_for_traffic_df['estimated_arrival'])])  # Using departure and estimated arrival times
.explode('custom_date', ignore_index = True))  # Explode the DataFrame based on the custom date range

hourly_exploded_scheduled_df


,id,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date
0,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 07:00:00
1,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 08:00:00
2,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 09:00:00
3,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 10:00:00
4,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 11:00:00
...,...,...,...,...,...,...,...,...,...
311568,1809,33788665,R-1ec2684e,2019-01-07 07:00:00,2019-01-08 02:03:36,0,2019-01-08 02:00:00,2019-01-07 07:00:00,2019-01-07 22:00:00
311569,1809,33788665,R-1ec2684e,2019-01-07 07:00:00,2019-01-08 02:03:36,0,2019-01-08 02:00:00,2019-01-07 07:00:00,2019-01-07 23:00:00
311570,1809,33788665,R-1ec2684e,2019-01-07 07:00:00,2019-01-08 02:03:36,0,2019-01-08 02:00:00,2019-01-07 07:00:00,2019-01-08 00:00:00
311571,1809,33788665,R-1ec2684e,2019-01-07 07:00:00,2019-01-08 02:03:36,0,2019-01-08 02:00:00,2019-01-07 07:00:00,2019-01-08 01:00:00


In [231]:
scheduled_traffic_df=hourly_exploded_scheduled_df.merge(traffic_df,left_on=['route_id','custom_date'], right_on=['route_id','date_time'],how='left')

In [232]:
scheduled_traffic_df

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date,id_y,date_time,no_of_vehicles,accident,time_period
0,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 07:00:00,1832681,2019-02-09 07:00:00,561.0,0,Morning
1,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 08:00:00,1832682,2019-02-09 08:00:00,2732.0,1,Morning
2,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 09:00:00,1832683,2019-02-09 09:00:00,2064.0,0,Late Morning
3,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 10:00:00,1832684,2019-02-09 10:00:00,2115.0,0,Late Morning
4,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,2019-02-09 16:00:00,2019-02-09 07:00:00,2019-02-09 11:00:00,1832685,2019-02-09 11:00:00,1881.0,0,Late Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311568,1809,33788665,R-1ec2684e,2019-01-07 07:00:00,2019-01-08 02:03:36,0,2019-01-08 02:00:00,2019-01-07 07:00:00,2019-01-07 22:00:00,1370231,2019-01-07 22:00:00,2359.0,0,Night
311569,1809,33788665,R-1ec2684e,2019-01-07 07:00:00,2019-01-08 02:03:36,0,2019-01-08 02:00:00,2019-01-07 07:00:00,2019-01-07 23:00:00,1370232,2019-01-07 23:00:00,2188.0,0,Night
311570,1809,33788665,R-1ec2684e,2019-01-07 07:00:00,2019-01-08 02:03:36,0,2019-01-08 02:00:00,2019-01-07 07:00:00,2019-01-08 00:00:00,1370233,2019-01-08 00:00:00,613.0,0,Late Night
311571,1809,33788665,R-1ec2684e,2019-01-07 07:00:00,2019-01-08 02:03:36,0,2019-01-08 02:00:00,2019-01-07 07:00:00,2019-01-08 01:00:00,1370234,2019-01-08 01:00:00,588.0,0,Late Night


In [233]:
def custom_agg(values):
    """
    Custom aggregation function to determine if any value in a group is 1 (indicating an accident).

    Args:
    values (iterable): Iterable of values in a group.

    Returns:
    int: 1 if any value is 1, else 0.
    """
    if any(values == 1):
        return 1
    else:
        return 0

In [234]:
scheduled_route_traffic_merge_df = scheduled_traffic_df.groupby(['id_x', 'truck_id', 'route_id'], as_index=False).agg(
                           avg_no_of_vehicles=('no_of_vehicles', 'mean'),
                           accident=('accident', custom_agg)
                       )

In [240]:
scheduled_route_traffic_merge_df

,id_x,truck_id,route_id,avg_no_of_vehicles,accident
0,1,30312694,R-b236e347,2240.428571,0
1,2,59856374,R-29ea762e,1837.272727,1
2,3,12602955,R-a3d67783,1966.000000,0
3,4,46619422,R-31ec9310,2225.571429,0
4,5,10140178,R-a07c5dbd,2206.800000,1
...,...,...,...,...,...
12303,12304,31047945,R-1484a7ea,1846.195652,1
12304,12305,14758432,R-927cf900,1837.450000,1
12305,12306,31370619,R-5a83ad98,1804.737113,1
12306,12307,67332883,R-991530bc,1820.306358,1


## Merge different merged tables

In [244]:
origin_destination_weather_traffic_merge=origin_destination_weather_merge_df.merge(scheduled_route_traffic_merge_df,on=['id_x','truck_id','route_id'],how='left')

In [263]:
schedule_weather_merge_df

,id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_description
0,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,76.000000,6.000000,58.000000,1011.666667,Sunny
1,9236,27121057,R-9e7ea58a,2019-02-12 07:00:00,2019-02-13 03:58:48,1,60.200000,5.200000,74.600000,1016.400000,Clear
2,1932,10966168,R-b2e25936,2019-01-10 07:00:00,2019-01-11 03:07:48,1,76.600000,12.000000,65.400000,1015.000000,Sunny
3,10256,11835599,R-8e99ef5f,2019-01-31 07:00:00,2019-02-01 16:31:48,1,NaN,NaN,NaN,NaN,NaN
4,5242,29121693,R-5efde222,2019-01-25 07:00:00,2019-01-25 12:01:48,0,45.666667,7.333333,64.000000,1015.000000,Partly cloudy
...,...,...,...,...,...,...,...,...,...,...,...
12303,10925,63052683,R-86dd80fc,2019-01-09 07:00:00,2019-01-11 08:04:48,1,72.500000,7.200000,62.600000,1012.600000,Clear
12304,2971,18678032,R-52653b9a,2019-01-13 07:00:00,2019-01-13 14:00:36,1,63.000000,11.333333,93.000000,1009.000000,Overcast
12305,9841,28817628,R-9b98b32f,2019-01-16 07:00:00,2019-01-17 16:22:12,1,43.714286,5.714286,84.142857,1019.571429,Clear
12306,7157,11759429,R-0bac03b2,2019-02-03 07:00:00,2019-02-03 20:16:48,0,84.500000,12.000000,45.750000,1009.750000,Sunny


In [265]:
origin_destination_weather_traffic_merge = origin_destination_weather_traffic_merge.drop(columns=['id_y'])

In [266]:
merged_data_weather_traffic=pd.merge(schedule_weather_merge_df, origin_destination_weather_traffic_merge, left_on=['id', 'truck_id', 'route_id', 'departure_date',
'estimated_arrival', 'delay'], right_on=['id_x', 'truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay'], how='left')

In [276]:
merged_data_weather_traffic_trucks = pd.merge(merged_data_weather_traffic, trucks_df, on='truck_id', how='left')

In [277]:
merged_data_weather_traffic_trucks = merged_data_weather_traffic_trucks.drop(columns='id')

In [278]:
final_merge = pd.merge(merged_data_weather_traffic_trucks, drivers_df, left_on='truck_id', right_on = 'vehicle_no', how='left')

In [279]:
def has_midnight(start, end):
    return int(start.date() != end.date())

In [280]:
final_merge['is_midnight'] = final_merge.apply(lambda row: has_midnight(row['departure_date'], row['estimated_arrival']), axis=1)

In [281]:
final_merge

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,8256,25645401,R-8adb758d,2019-02-09 07:00:00,2019-02-09 15:31:12,0,76.000000,6.000000,58.000000,1011.666667,...,adaa60e8-1,Charles Adams,male,43.0,4.0,proactive,4.0,25645401.0,60.74,0
1,9236,27121057,R-9e7ea58a,2019-02-12 07:00:00,2019-02-13 03:58:48,1,60.200000,5.200000,74.600000,1016.400000,...,39a9b538-9,Tony Cantu,male,47.0,11.0,conservative,4.0,27121057.0,44.85,1
2,1932,10966168,R-b2e25936,2019-01-10 07:00:00,2019-01-11 03:07:48,1,76.600000,12.000000,65.400000,1015.000000,...,71c1bcb0-b,Leonard Austin,male,45.0,8.0,conservative,3.0,10966168.0,43.40,1
3,10256,11835599,R-8e99ef5f,2019-01-31 07:00:00,2019-02-01 16:31:48,1,NaN,NaN,NaN,NaN,...,102f88eb-7,Joseph Ortega,male,47.0,15.0,conservative,5.0,11835599.0,38.61,1
4,5242,29121693,R-5efde222,2019-01-25 07:00:00,2019-01-25 12:01:48,0,45.666667,7.333333,64.000000,1015.000000,...,40bc7695-3,Craig Day,male,47.0,9.0,proactive,8.0,29121693.0,60.34,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10925,63052683,R-86dd80fc,2019-01-09 07:00:00,2019-01-11 08:04:48,1,72.500000,7.200000,62.600000,1012.600000,...,d52284fa-8,Adam Turner,male,50.0,16.0,conservative,7.0,63052683.0,46.99,1
12304,2971,18678032,R-52653b9a,2019-01-13 07:00:00,2019-01-13 14:00:36,1,63.000000,11.333333,93.000000,1009.000000,...,3bdccbfa-b,Dustin Jennings,male,45.0,16.0,proactive,9.0,18678032.0,59.91,0
12305,9841,28817628,R-9b98b32f,2019-01-16 07:00:00,2019-01-17 16:22:12,1,43.714286,5.714286,84.142857,1019.571429,...,50f948b3-0,Timothy Barrera,male,48.0,9.0,proactive,2.0,28817628.0,56.51,1
12306,7157,11759429,R-0bac03b2,2019-02-03 07:00:00,2019-02-03 20:16:48,0,84.500000,12.000000,45.750000,1009.750000,...,2650aff1-2,David Santana,male,45.0,17.0,conservative,8.0,11759429.0,44.85,0


In [275]:
final_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_description',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours',
       'date_time_x', 'city_id_x', 'temp_x', 'wind_speed_x', 'description_x',
       'humidity_x', 'pressure_x', 'id_y', 'date_time_y', 'city_id_y',
       'temp_y', 'wind_speed_y', 'description_y', 'humidity_y', 'pressure_y',
       'avg_no_of_vehicles', 'accident', 'id_x', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'id_y', 'driver_id',
       'name', 'gender', 'age', 'experience', 'driving_style', 'ratings',
       'vehicle_no', 'average_speed_mph', 'is_midnight'],
      dtype='object')